In [ ]:
# 統合報告書表彰　企業別ランキング集計
# 各社の集計結果を比較可能なポイントに変換・集計し、企業を順位付け

# ポイント化する指標は、
# ・知財関連KWの種類
# ・知財関連KWの件数
# ・知財関連KWの相対的登場頻度
# ・知財関連KW×金融関連KWの共起の種類
# ・知財関連KW×金融関連KW件数
# ・知財関連KW×金融関連KWの相対的登場頻度
# ・金融関連KWの中心性指標の登場頻度
# 
# 対象KW等の抽出、件数等の集計はBigQueryで実行、結果をPythonのデータフレームに格納後、順位付けに使用するポイントを付与
# 複数のデータフレームに格納した指標・ポイントを各々Excelのワークシートに格納、


In [ ]:
from google.cloud import bigquery
import pandas as pd


client = bigquery.Client()

In [ ]:
#知財KWの一覧

sql_1 = """
SELECT
    dt.`コード` AS `コード`,
    dt.`企業名` AS `企業名`,
    dt.`業種` AS `業種`,
    dt.`分析範囲` AS `分析範囲`,
    dt.`単語` AS `知財関連KW`,
    dt.`件数` AS `件数`,
    dt.`件数_Rank` AS `件数_Rank`

 FROM
     `my-ip-project-425607.LESJ_Textmining.Merged_WordList_20250520` dt

 LEFT JOIN 
    `my-ip-project-425607.LESJ_Textmining.知的資本関連KW_20250518` k1
ON 
    dt.`単語` = k1.`Keyword`

WHERE
    (k1.keyword IS NOT NULL) AND (dt.`分析範囲` = '経営者+価値創造+知的資本')
    
ORDER BY
    `コード`,
    `件数` DESC ;

"""

df_IP_KW_List = client.query(sql_1).to_dataframe()
# print(df_IP_KW_List)


In [ ]:
#知財KWの件数

sql = """
SELECT
    dt.`コード` AS `コード`,
    dt.`企業名` AS `企業名`,
    dt.`分析範囲` AS `分析範囲`,
    SUM(dt.`件数`)  AS `総単語件数`,

 FROM
     `my-ip-project-425607.LESJ_Textmining.Merged_WordList_20250520` dt

 LEFT JOIN 
    `my-ip-project-425607.LESJ_Textmining.知的資本関連KW_20250518` k1
ON 
    dt.`単語` = k1.`Keyword`

WHERE
    (k1.keyword IS NOT NULL) AND (dt.`分析範囲` = '経営者+価値創造+知的資本')
    
GROUP BY
    `コード`, `企業名`, `分析範囲`
ORDER BY
    `総単語件数` DESC ;

"""

df_Num_of_IP_KW = client.query(sql).to_dataframe()
# print(df_IP_KW_List)


In [ ]:
#知財KWの種類数

sql = """
SELECT
    dt.`コード` AS `コード`,
    dt.`企業名` AS `企業名`,
    dt.`分析範囲` AS `分析範囲`,
    COUNT(dt.`単語`)  AS `単語種類数`,

 FROM
     `my-ip-project-425607.LESJ_Textmining.Merged_WordList_20250520` dt

 LEFT JOIN 
    `my-ip-project-425607.LESJ_Textmining.知的資本関連KW_20250518` k1
ON 
    dt.`単語` = k1.`Keyword`

WHERE
    (k1.keyword IS NOT NULL) AND (dt.`分析範囲` = '経営者+価値創造+知的資本')
    
GROUP BY
    `コード`, `企業名`, `分析範囲`
ORDER BY
    `単語種類数` DESC ;
"""

df_Num_of_Types_of_IP_KW = client.query(sql).to_dataframe()
print(df_IP_KW_Num_of_Types)


In [17]:
#知財KW×金融KWの共起リスト

sql = """
SELECT 
    dt.`コード` AS `コード`,
    dt.`企業名` AS `企業名`,
    dt.`分析範囲` AS `分析範囲`,
    dt.`1番目` AS `1番目`,
    dt.`2番目` AS `2番目`,
    dt.count,
    dt.count_Rank,

FROM 
    `my-ip-project-425607.LESJ_Textmining.Merged_Co-Occurrence_20250520` dt
LEFT JOIN 
    `my-ip-project-425607.LESJ_Textmining.知的資本関連KW_20250518` k1
ON 
    dt.`1番目` = k1.keyword OR dt.`2番目` = k1.keyword
LEFT JOIN 
    `my-ip-project-425607.LESJ_Textmining.金融関連KW_20250518` k2
ON 
    dt.`1番目` = k2.keyword OR dt.`2番目` = k2.keyword
WHERE
    (k1.keyword IS NOT NULL) AND (k2.keyword IS NOT NULL) 
    AND (dt.`分析範囲` = '経営者+価値創造+知的資本')
ORDER BY
    `コード`,
    dt.count DESC;

"""

df_List_of_Co_Occurence = client.query(sql).to_dataframe()
print(df_List_of_Co_Occurence)


        コード         企業名           分析範囲      1番目   2番目  count  count_Rank
0     13770      サカタのタネ  経営者+価値創造+知的資本      売上高  研究開発      3         329
1     13770      サカタのタネ  経営者+価値創造+知的資本       投資  研究開発      2         737
2     13770      サカタのタネ  経営者+価値創造+知的資本     事業戦略    収益      2         737
3     13770      サカタのタネ  経営者+価値創造+知的資本  ビジネスモデル  事業戦略      1        2364
4     13770      サカタのタネ  経営者+価値創造+知的資本  ビジネスモデル  自社開発      1        2364
...     ...         ...            ...      ...   ...    ...         ...
2690  96130  ＮＴＴデータグループ  経営者+価値創造+知的資本       売上  経営戦略      1        9559
2691  96130  ＮＴＴデータグループ  経営者+価値創造+知的資本      アプリ    銀行      1        9559
2692  96130  ＮＴＴデータグループ  経営者+価値創造+知的資本     事業戦略    投資      1        9559
2693  96130  ＮＴＴデータグループ  経営者+価値創造+知的資本       契約  金融機関      1        9559
2694  96130  ＮＴＴデータグループ  経営者+価値創造+知的資本     成長戦略    金融      1        9559

[2695 rows x 7 columns]


In [19]:
#知財KW×金融KWの共起件数

sql = """
SELECT 
    dt.`コード` AS `コード`,
    dt.`企業名` AS `企業名`,
    dt.`分析範囲` AS `分析範囲`,
    SUM(dt.count) AS `共起件数`,

FROM 
    `my-ip-project-425607.LESJ_Textmining.Merged_Co-Occurrence_20250520` dt
LEFT JOIN 
    `my-ip-project-425607.LESJ_Textmining.知的資本関連KW_20250518` k1
ON 
    dt.`1番目` = k1.keyword OR dt.`2番目` = k1.keyword
LEFT JOIN 
    `my-ip-project-425607.LESJ_Textmining.金融関連KW_20250518` k2
ON 
    dt.`1番目` = k2.keyword OR dt.`2番目` = k2.keyword
WHERE
    (k1.keyword IS NOT NULL) AND (k2.keyword IS NOT NULL) 
    AND (dt.`分析範囲` = '経営者+価値創造+知的資本')

GROUP BY 
    `コード`,`企業名`,`分析範囲`

ORDER BY 
    `共起件数` DESC;

"""

df_Num_of_Co_Occurrence = client.query(sql).to_dataframe()
# Rankを付与
df_Num_of_Co_Occurrence['共起件数_Rank'] = df_Num_of_Co_Occurrence['共起件数'].rank(method='dense', ascending=False).astype(int)
# 得点を付与
df_Num_of_Co_Occurrence['共起件数_点数'] = (10 - ((df_Num_of_Co_Occurrence['共起件数_Rank'] - 1) // 10)).clip(lower=1)

print(df_Num_of_Co_Occurrence)


       コード      企業名           分析範囲  共起件数  共起件数_Rank  共起件数_点数
0    49110      資生堂  経営者+価値創造+知的資本   140          1       10
1    34070      旭化成  経営者+価値創造+知的資本   137          2       10
2    69020     デンソー  経営者+価値創造+知的資本   133          3       10
3    38800     大王製紙  経営者+価値創造+知的資本   120          4       10
4    67580  ソニーグループ  経営者+価値創造+知的資本   120          4       10
..     ...      ...            ...   ...        ...      ...
122  57270  東邦チタニウム  経営者+価値創造+知的資本     2         65        4
123  80150     豊田通商  経営者+価値創造+知的資本     2         65        4
124  62010   豊田自動織機  経営者+価値創造+知的資本     2         65        4
125  40630   信越化学工業  経営者+価値創造+知的資本     2         65        4
126  70110    三菱重工業  経営者+価値創造+知的資本     1         66        4

[127 rows x 6 columns]


In [21]:
#知財KW×金融KWの共起種類数

sql = """
SELECT 
    dt.`コード` AS `コード`,
    dt.`企業名` AS `企業名`,
    dt.`分析範囲` AS `分析範囲`,
    count(dt.`企業名`) AS `共起種類数`,

FROM 
    `my-ip-project-425607.LESJ_Textmining.Merged_Co-Occurrence_20250520` dt
LEFT JOIN 
    `my-ip-project-425607.LESJ_Textmining.知的資本関連KW_20250518` k1
ON 
    dt.`1番目` = k1.keyword OR dt.`2番目` = k1.keyword
LEFT JOIN 
    `my-ip-project-425607.LESJ_Textmining.金融関連KW_20250518` k2
ON 
    dt.`1番目` = k2.keyword OR dt.`2番目` = k2.keyword
WHERE
    (k1.keyword IS NOT NULL) AND (k2.keyword IS NOT NULL) 
    AND (dt.`分析範囲` = '経営者+価値創造+知的資本')

GROUP BY `コード`,`企業名`,`分析範囲`

ORDER BY `共起種類数` DESC
;

"""

df_Num_of_Types_of_Co_Occurrence = client.query(sql).to_dataframe()
# Rankを付与
df_Num_of_Types_of_Co_Occurrence['共起種類数_Rank'] = df_Num_of_Types_of_Co_Occurrence['共起種類数'].rank(method='dense', ascending=False).astype(int)
# 得点を付与
df_Num_of_Types_of_Co_Occurrence['共起種類数_点数'] = (10 - ((df_Num_of_Types_of_Co_Occurrence['共起種類数_Rank'] - 1) // 10)).clip(lower=1)
print(df_Num_of_Types_of_Co_Occurrence)


       コード             企業名           分析範囲  共起種類数  共起種類数_Rank  共起種類数_点数
0    34070             旭化成  経営者+価値創造+知的資本     81           1        10
1    41830            三井化学  経営者+価値創造+知的資本     71           2        10
2    67580         ソニーグループ  経営者+価値創造+知的資本     65           3        10
3    69020            デンソー  経営者+価値創造+知的資本     59           4        10
4    46120  日本ペイントホールディングス  経営者+価値創造+知的資本     57           5        10
..     ...             ...            ...    ...         ...       ...
122  40630          信越化学工業  経営者+価値創造+知的資本      2          43         6
123  62010          豊田自動織機  経営者+価値創造+知的資本      2          43         6
124  53100            東洋炭素  経営者+価値創造+知的資本      2          43         6
125  57270         東邦チタニウム  経営者+価値創造+知的資本      2          43         6
126  70110           三菱重工業  経営者+価値創造+知的資本      1          44         6

[127 rows x 6 columns]


In [ ]:
#知財KWの中心性指標

sql = """
SELECT
	dt.`コード` AS `コード`,
	dt.`企業名` AS `企業名`,
	dt.`分析範囲` AS `分析範囲`,
	dt.`Node` AS `Node`,
	dt.`Degree Centrality` AS `Degree Centrality`,
	dt.`Betweenness Centrality` AS `Betweenness Centrality`,
	dt.`Closeness Centrality` AS `Closeness Centrality`,
	dt.`Eigenvector Centrality` AS `Eigenvector Centrality`,
	dt.`Community` AS `Community`,
	dt.`Degree Centrality_Rank` AS `Degree Centrality_Rank`,
	dt.`Betweenness Centrality_Rank` AS `Betweenness Centrality_Rank`,
	dt.`Closeness Centrality_Rank` AS `Closeness Centrality_Rank`,
	dt.`Eigenvector Centrality_Rank` AS `Eigenvector Centrality_Rank`


 FROM
     my-ip-project-425607.LESJ_Textmining.Merged_Centrality_20250520 dt

 LEFT JOIN 
    `my-ip-project-425607.LESJ_Textmining.知的資本関連KW_20250518` k1
ON 
    dt.`Node` = k1.`Keyword`

WHERE
    (k1.keyword IS NOT NULL) AND (dt.`分析範囲` = '経営者+価値創造+知的資本')
    
ORDER BY
    `コード`,
    `Degree Centrality` DESC ;


"""

df_Centrality = client.query(sql).to_dataframe()
print(df_Centrality)
